# Day 28: Bias Mitigation (In-Processing)

In this lab, we will use `AdversarialDebiaser` to mitigate bias in our model predictions.
We will simulate a scenario where a model produces biased scores for two groups, and then we will align their distributions.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.fairness.mitigation import AdversarialDebiaser
from src.fairness.metrics import FairnessEvaluator

## 1. Simulate Biased Scores

We generate scores where 'Group A' has a higher mean than 'Group B'.

In [ ]:
np.random.seed(0)
n = 1000
groups = np.array(['Group A'] * 500 + ['Group B'] * 500)

# Biased scores
# Group A: Mean 0.7, Std 0.1
# Group B: Mean 0.4, Std 0.1
scores = np.concatenate([
    np.random.normal(0.7, 0.1, 500),
    np.random.normal(0.4, 0.1, 500)
])
scores = np.clip(scores, 0, 1)

# Predictions at threshold 0.5
preds = (scores > 0.5).astype(int)

# Measure initial fairness
dp_diff = FairnessEvaluator.demographic_parity_difference(preds, groups)
print(f"Initial DP Difference: {dp_diff:.4f}")

## 2. Apply Mitigation

We use the `AdversarialDebiaser` to align the score distributions.

In [ ]:
mitigator = AdversarialDebiaser()
scores_debiased = mitigator.fit_transform(scores, groups)

# New predictions
preds_debiased = (scores_debiased > 0.5).astype(int)

# Measure new fairness
dp_diff_new = FairnessEvaluator.demographic_parity_difference(preds_debiased, groups)
print(f"New DP Difference: {dp_diff_new:.4f}")

## 3. Visualize Distributions

Let's see how the histograms of scores have changed.

In [ ]:
results = pd.DataFrame({
    'Group': groups,
    'Original Score': scores,
    'Debiased Score': scores_debiased
})

print("Original Means:")
print(results.groupby('Group')['Original Score'].mean())
print("\nDebiased Means:")
print(results.groupby('Group')['Debiased Score'].mean())